# Auto-ablation with Howso Engine

> Note: This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using auto-ablation during train to reduce the size of your data _as they are trained_.

In [1]:
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Load Data & Feature Mapping

Our example dataset for this recipe is the well-known `Adult` dataset. This dataset works well because it has over 48,000 cases. In general, datasets containing over 25,000 rows work well. The default minimum size is 1,000 cases, below which no cases will be ablated.

This part of the process is identical to other recipes.

In [ ]:
df = fetch_data("adult", local_cache_dir="../../../data")
features = infer_feature_attributes(df)

display(df)
display(features.to_dataframe())

## Step 3: Create Trainee and Set Parameters

The process of creating the `Trainee` is the same as the other recipes. However, after the `Trainee` is created, we set auto-analyze and auto-ablation parameters to ensure that the `Trainee` remains informed as it removes cases during train.

In [3]:
t = Trainee(features=features)

# Set auto-analyze parameters. Note that use_case_weights=True since case-weights are how
# information from ablated cases is retained in the Trainee.
t.set_auto_analyze_params(auto_analyze_enabled=True, use_case_weights=True)

# Set auto-ablation parameters.
t.set_auto_ablation_params(
    auto_ablation_enabled=True,
    influence_weight_entropy_threshold=0.6,
    min_num_cases=2_500,
    max_num_cases=10_000,
    delta_threshold_map={
        "accuracy": {"target": 0.1}
    }
)

This recipe uses the following parameters for auto-ablation:

- `auto_ablation_enabled` — When this parameter is `False`, auto-ablation will not be performed.
- `influence_weight_entropy_threshold` — This parameter is used to determine which cases will be ablated.
  If the entropy of a case's influence weights would be above this percentile of existing cases' influence
  weight entropies, then the case is ablated. This is set to $0.6$, the default value.
- `min_num_cases` — This parameter sets the minimum number of cases that are allowed to be in the Trainee.
- `max_num_cases` — This parameter sets the maximum number of cases that are allowed to be in the Trainee.
- `delta_threshold_map` — This parameter defines thresholds that are used to determine when ablation should stop.
  In this case, the threshold map instructs auto-ablation to stop if the delta between the current accuracy
  of the `"target"` feature and its previous accuracy is greater than $0.1$. I.e., ablation will stop if accuracy
  for that feature drops from $0.8$ to $0.7$. More formally, ablation will stop if
  $\text{acc}_\text{old} - \text{acc}_\text{new} \geq 0.1$.

## Step 4: Training

Now that the parameters are set, the `Trainee` can be trained as if this were a normal workflow.

In [4]:
t.train(df)

Notably, only a subset of the cases have been trained (the rest have been ablated).

In [ ]:
num_trained_cases = t.get_num_training_cases()
num_total_cases = len(df)
num_trained_percent = round((num_trained_cases / num_total_cases) * 100, 3)

print(f"Trained {num_trained_cases:,} cases out of {num_total_cases:,} ({num_trained_percent}%).")

## Step 5: Results

An ablated `Trainee` can be used in much the same way as other `Trainee`s, as long as `use_case_weights=True`. Here, we will investigate the prediction stats for all features.

In [6]:
stats = t.get_prediction_stats(
    action_feature="target",
    details={
        "prediction_stats": True,
        "selected_prediction_stats": ["all"],
    },
)

In [ ]:
stats.loc[stats.index != "confusion_matrix", :]

In [ ]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix for 'target'")
matrix = pd.DataFrame(stats["target"]["confusion_matrix"]["matrix"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

## Conclusion

Howso Engine with auto-ablation can compress data to a fraction of its original size while ensuring accuracy (and other measures) remain in an acceptable range without compromising the other capabilities of the `Trainee`.